# RAW DATA


In [0]:
# Read the Delta table from the specified storage location
delta_table_path = "/mnt/prod_rawdata/hackathon"
review_df = spark.read.format("delta").load(delta_table_path)

# Display the DataFrame
display(review_df)

# INITIAL TRANSFORMATIONS AND LINKAGE 

In [0]:
%sql
SELECT * FROM default.trustpilot_review_data

# SENTIMENTAL ANALYSIS

## Hugging Face PreTrained model

In [0]:
# Import hugging face libraries and pandas
from transformers import pipeline
import pandas as pd
#Convert the DataFrame to a pandas DataFrame
review_df = spark.sql("SELECT * FROM default.trustpilot_review_data where length(TRANSLATION)<=512").toPandas()

## Feed data to analyzer 


In [0]:
def analyze_sentiment(review):
    result = sentiment_analyzer(review)[0]
    return pd.Series([result['label'], result['score']])


review_df[['sentiment', 'score']] = review_df['TRANSLATION'].apply(analyze_sentiment)
review_spark_df = spark.createDataFrame(review_df)


## Save data to table

In [0]:
 review_spark_df.write.mode('overwrite').saveAsTable('default.trustpilot_review_data_sentiment')

## REFINE SENTIMENTS TO ALIGN WITH LEBARA


### Normalize Sentiment

In [0]:
%sql
update default.trustpilot_review_data_sentiment 
set normalized_sentiment = 'NEGATIVE'
where sentiment = 'POSITIVE'
and stars in(2 ,1 )
and TRANSLATION = 'n/a'

In [0]:
%sql 
update default.trustpilot_review_data_sentiment 
set normalized_sentiment = 'NEUTRAL'
where 1 = 1 --sentiment = 'POSITIVE'
and stars = 3 
and TRANSLATION = 'n/a'

In [0]:
%sql 
-- negative star review preceeds the positive sentiment in general 
update default.trustpilot_review_data_sentiment 
set normalized_sentiment = 'NEGATIVE'
where sentiment = 'POSITIVE'
and stars = 1 

In [0]:
%sql 
-- for leabar cheap is good 
update default.trustpilot_review_data_sentiment 
set normalized_sentiment = 'POSITIVE'
where sentiment = 'NEGATIVE'
and stars in ( 4 , 5)
and translation like '%cheap%'

In [0]:
%sql 
update default.trustpilot_review_data_sentiment 
set normalized_sentiment = 'POSITIVE'
where sentiment = 'NEGATIVE'
and stars in ( 4 , 5)
and translation like '%good%'
and NORMALIZED_SENTIMENT is null 

In [0]:
%sql 
update default.trustpilot_review_data_sentiment 
set normalized_sentiment = 'POSITIVE'
where sentiment = 'NEGATIVE'
and stars in ( 4 , 5)
and translation like '%good%'
and NORMALIZED_SENTIMENT is null 

In [0]:

%sql 
update default.trustpilot_review_data_sentiment 
set normalized_sentiment = 'NEGATIVE'
where sentiment = 'POSITIVE'
and stars in(2 ,1 )
and TRANSLATION = 'n/a'

#DATA ENRICHMENT

-  - Add Churn Score 
-  - Average Customer Value 
-  - Pre paid vs Post paid 
-  - Tenure 
-  - Customer Type 

In [0]:
review_df = spark.sql("select * from default.trustpilot_review_data_sentiment")
review_df = review_df.join(churn_df_final, 'SUBSCRIBER_SR_KEY', 'left').select(review_df["*"], churn_df_final["PREDICTION_SCORE"].alias('CHURN_PREDICTION_SCORE'))
review_df = review_df.withColumn('PREDICTION_SCORE', F.coalesce(F.col('CHURN_PREDICTION_SCORE'), F.col('PREDICTION_SCORE')))
review_df = review_df.drop('CHURN_PREDICTION_SCORE')
review_df = review_df.withColumn("CUSTOMER_TYPE",F.when(F.col('LAST_CUSTOMER_TYPE_SR_KEY')==1, F.lit("POSTPAID")).when(F.col('LAST_CUSTOMER_TYPE_SR_KEY').isin([2,4,5]), F.lit("PREPAID")).otherwise(F.lit("UNKNOWN")))
review_df = review_df.withColumn("NORMALIZED_SENTIMENT", F.lit(None))
review_df.write.mode('overwrite').saveAsTable('default.trustpilot_review_data_sentiment')

# KEYWORD EXTRACTION


In [0]:
# install libraries
%pip install keybert

### Function to extract keywords

In [0]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
kw_model = KeyBERT(model=sentence_model)

In [0]:
def extract_keyword(review):
    keywords = kw_model.extract_keywords(review, keyphrase_ngram_range=(3, 3), stop_words='english',
                              use_mmr=True, diversity=0.2)
    
    try:
      val = pd.Series([list(keywords[0])[0], list(keywords[1])[0], list(keywords[2])[0], list(keywords[3])[0], list(keywords[4])[0]])
      return val
    except:
      return pd.Series([None, None, None, None, None])

In [0]:
#Convert table  to pandas dataframe
review_df = spark.sql('select REVIEW_ID, TRANSLATION from trustpilot_review_data_sentiment').toPandas()

In [0]:
review_df[['KEYWORD_1', 'KEYWORD_2', 'KEYWORD_3', 'KEYWORD_4', 'KEYWORD_5']] = review_df['TRANSLATION'].apply(extract_keyword)
review_spark_df = spark.createDataFrame(review_df)
review_spark_df.write.mode('overwrite').saveAsTable('default.trustpilot_review_data_with_keyword')

In [0]:
from pyspark.sql import functions as F
keyword_df = spark.table('trustpilot_review_data_with_keyword')
keyword_df = keyword_df.filter(F.col('KEYWORD_1').isNotNull())
review_df = spark.table('trustpilot_review_data_sentiment')
keyword_df = keyword_df.selectExpr("REVIEW_ID", "TRANSLATION","stack(5, 'KEYWORD_1', KEYWORD_1, 'KEYWORD_2', KEYWORD_2, 'KEYWORD_3', KEYWORD_3, 'KEYWORD_4', KEYWORD_4, 'KEYWORD_5', KEYWORD_5) as (KEYWORD,value)")
unpivoted_keyword_df = keyword_df.drop('KEYWORD').withColumnRenamed('value','KEYWORD')
unpivoted_keyword_final_df = unpivoted_keyword_df.join(review_df, 'REVIEW_ID').select(unpivoted_keyword_df['REVIEW_ID'], review_df['COUNTRY_CODE'], review_df["STARS"],  unpivoted_keyword_df['KEYWORD'], review_df['EXPERIENCED_AT'])
unpivoted_keyword_final_df.write.mode('overwrite').saveAsTable('default.trustpilot_review_data_stars_with_keyword')

## Final Keyword extracted data

In [0]:
%sql
select * from trustpilot_review_data_stars_with_keyword--where STARS=1 group by KEYWORD